In [2]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")  # Qdrant endpointini girin

collection_name = "hybrid_search"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(size=384, distance=models.Distance.COSINE),
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(on_disk=True)
        )
    }
)


/tmp/ipykernel_119197/2716592562.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [3]:
from datasets import load_dataset

ds = load_dataset("umarigan/turkish_wikipedia")

In [4]:
import pandas as pd

wiki_data = ds["train"].select(range(0, 10000))


In [5]:
print(wiki_data)

Dataset({
    features: ['id', 'text', 'title'],
    num_rows: 10000
})


In [6]:
id = list(wiki_data["id"])
title = list(wiki_data["title"])
text = list(wiki_data["text"])

In [7]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


cleaned_texts = [clean_text(t) for t in text]


In [8]:
print(cleaned_texts[0])

film şu anlamlara gelebilir camlara yapıştırılarak içerinin görünmesini engelleyen bir tür ince yaprak sinemacılıkta bir oyunun bütününü taşıyan şerit veya şeritlerin bütünü film fotoğrafçılık fotoğrafçılıkta radyografide ve sinemacılıkta resim çekmek için kullanılan selülozdan saydam bükülebilir şerit film sinema sinema makinesiyle gösterilen eser izleti film film samuel beckett in yazdığı tek senaryodan çekilen 1965 abd yapımı film total film 1997 den beri i ngiltere de yayımlanan sinema dergisi


In [9]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import tiktoken
import re

def chunk_text(text, max_tokens=128):
    """
    Split text into chunks of maximum 128 tokens.
    
    Args:
        text (str): Text to chunk
        max_tokens (int): Maximum tokens per chunk (default: 128)
    
    Returns:
        list: List of text chunks
    """
    # Initialize tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # If text is already small enough, return as is
    if len(tokenizer.encode(text)) <= max_tokens:
        return [text]
    
    # Split by sentences first
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Test if adding this sentence exceeds limit
        test_chunk = current_chunk + (" " if current_chunk else "") + sentence
        
        if len(tokenizer.encode(test_chunk)) <= max_tokens:
            current_chunk = test_chunk
        else:
            # Save current chunk if it has content
            if current_chunk:
                chunks.append(current_chunk.strip())
            
            # Start new chunk with current sentence
            # If single sentence is too long, split by words
            if len(tokenizer.encode(sentence)) > max_tokens:
                words = sentence.split()
                temp_chunk = ""
                
                for word in words:
                    test_word_chunk = temp_chunk + (" " if temp_chunk else "") + word
                    
                    if len(tokenizer.encode(test_word_chunk)) <= max_tokens:
                        temp_chunk = test_word_chunk
                    else:
                        if temp_chunk:
                            chunks.append(temp_chunk.strip())
                        temp_chunk = word
                
                current_chunk = temp_chunk
            else:
                current_chunk = sentence
    
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def chunk_dataset(dataset, max_tokens=128):
    """
    Chunk a list of texts.
    
    Args:
        dataset (list): List of text paragraphs
        max_tokens (int): Maximum tokens per chunk
    
    Returns:
        list: All chunks from all paragraphs
    """
    all_chunks = []
    for text in dataset:
        chunks = chunk_text(text, max_tokens)
        all_chunks.extend(chunks)
    return all_chunks


In [11]:
cleaned_texts=chunk_dataset(cleaned_texts)

In [12]:
for i in range (0,11):
    print(cleaned_texts[i])

film şu anlamlara gelebilir camlara yapıştırılarak içerinin görünmesini engelleyen bir tür ince yaprak sinemacılıkta bir oyunun bütününü taşıyan şerit veya şeritlerin bütünü film fotoğrafçılık fotoğrafçılıkta radyografide ve sinemacılıkta resim çekmek için kullanılan selülozdan saydam bükülebilir şerit film sinema sinema
makinesiyle gösterilen eser izleti film film samuel beckett in yazdığı tek senaryodan çekilen 1965 abd yapımı film total film 1997 den beri i ngiltere de yayımlanan sinema dergisi
bolşevik çoğunluktan yana anlamına gelen rusça kelime 1903 yılında düzenlenen rusya sosyal demokrat i şçi partisi nin i kinci kongresi nde vladimir lenin ve julius martov arasında yeni kurulmakta olan partinin üyelik tanımı üzerine başlayan görüş ayrılığı sonucu yaşanan ayrışmadaki taraflardan lenin yanlısı grup kongrede lenin yanlıları
çoğunlukta olduğu için rusça çoğunluk anlamına gelen bolşevik olarak azınlıktaki martov yanlıları da menşevik olarak adlandırılacaktır kongreden sonra iki tar

In [13]:
from sentence_transformers import SentenceTransformer
texts = cleaned_texts
dense_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device="cuda")
dense_vectors = dense_model.encode(texts, show_progress_bar=True, batch_size=128).tolist()

/home/alicantanyeri/.local/lib/python3.13/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/alicantanyeri/.local/lib/python3.13/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Batches: 100%|██████████| 1437/1437 [13:14<00:00,  1.81it/s]


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_sparse_matrix = tfidf.fit_transform(cleaned_texts)

In [15]:
import pickle

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [16]:
def to_sparse_vec(sparse_row):
    indices = sparse_row.indices.tolist()
    values = sparse_row.data.tolist()
    return {"indices": indices, "values": values}



In [17]:
from qdrant_client.models import PointStruct, SparseVector

batch_size = 250  
points = []

for start_idx in range(0, len(texts), batch_size):
    end_idx = min(start_idx + batch_size, len(texts))

    batch_points = []
    for idx in range(start_idx, end_idx):
        dense_vec = dense_vectors[idx]
        sparse_dict = to_sparse_vec(tfidf_sparse_matrix[idx])
        sparse_vec = SparseVector(indices=sparse_dict["indices"], values=sparse_dict["values"])

        batch_points.append(
            PointStruct(
                id=idx,
                vector={
                    "dense": dense_vec,
                    "sparse": sparse_vec
                },
                payload={
                    "id": id[idx],
                    "text": wiki_data["text"][idx],
                    "title": title[idx]}
            )
        )
    
    client.upsert(collection_name=collection_name, points=batch_points)
    print(f"Batch {start_idx} - {end_idx} yüklendi.")


Batch 0 - 250 yüklendi.
Batch 250 - 500 yüklendi.
Batch 500 - 750 yüklendi.
Batch 750 - 1000 yüklendi.
Batch 1000 - 1250 yüklendi.
Batch 1250 - 1500 yüklendi.
Batch 1500 - 1750 yüklendi.
Batch 1750 - 2000 yüklendi.
Batch 2000 - 2250 yüklendi.
Batch 2250 - 2500 yüklendi.
Batch 2500 - 2750 yüklendi.
Batch 2750 - 3000 yüklendi.
Batch 3000 - 3250 yüklendi.
Batch 3250 - 3500 yüklendi.
Batch 3500 - 3750 yüklendi.
Batch 3750 - 4000 yüklendi.
Batch 4000 - 4250 yüklendi.
Batch 4250 - 4500 yüklendi.
Batch 4500 - 4750 yüklendi.
Batch 4750 - 5000 yüklendi.
Batch 5000 - 5250 yüklendi.
Batch 5250 - 5500 yüklendi.
Batch 5500 - 5750 yüklendi.
Batch 5750 - 6000 yüklendi.
Batch 6000 - 6250 yüklendi.
Batch 6250 - 6500 yüklendi.
Batch 6500 - 6750 yüklendi.
Batch 6750 - 7000 yüklendi.
Batch 7000 - 7250 yüklendi.
Batch 7250 - 7500 yüklendi.
Batch 7500 - 7750 yüklendi.
Batch 7750 - 8000 yüklendi.
Batch 8000 - 8250 yüklendi.
Batch 8250 - 8500 yüklendi.
Batch 8500 - 8750 yüklendi.
Batch 8750 - 9000 yüklendi.
B

IndexError: list index out of range